In [1]:
%pip install --upgrade baolib


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import tempfile
from pathlib import Path

from baolib.bao import (
    Access,
    AccessChange,
    Vault,
    DB,
    Store,
    Replica,
    Message,
    Mailbox,
    newPrivateID,
    publicID,
    decodeID,
    local_store,
    set_bao_log_level,
)

set_bao_log_level('error')

/Users/ea/Documents/Lab/bao/bindings/py/baolib/baod.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
INFO  [05:38:10] (+20ms) core.syncTime[time.go:34] - clock offset 11.170456ms from time.google.com 


AttributeError: dlsym(0x737b57c0, bao_listGroups): symbol not found

# Introduction
Bao is a secure storage library for sharing encrypted data. This notebook shows the Python binding (`baolib`) API.


## Identity
Create identities with `newPrivateID()` and derive the public ID with `publicID()`.


In [3]:
bob = newPrivateID()
bob_public = publicID(bob)

print('Owner public ID:', bob_public)

keys = decodeID(bob)
print(f'secp256k1 key: {keys["cryptKey"]}, ed25519 key: {keys["signKey"]}') 


Owner public ID: A_1HvChUzrsb93445GF5w8t7a9mlEjMXC0PvfQJTJaL6t4duVKvuW34SNuS3S1Kd9SUAlrkqIX6XxnO4eKtCvCE=
secp256k1 key: HEQSDDzscPTfF1J54RRYYEyrD6Uqe0OVsFR0RDG8yvc=, ed25519 key: 0ckOzq1OvTh6sq4jC80_PTNTwMSKBio9ct2DYyaU0zE=


## Local database
Use a local SQLite database to track metadata. Here we start with a clean DB in a temp directory.


In [4]:
temp_root = Path(tempfile.gettempdir()) / 'bao_python_sample'
temp_root.mkdir(parents=True, exist_ok=True)

def new_db_path(name: str) -> DB:
    db_path = temp_root / name
    if db_path.exists():
        db_path.unlink()
    return str(db_path)

db_path = new_db_path('bao.db')
db = DB('sqlite3', db_path)
print('Database at', db_path)


Database at /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/bao.db


## Create a Bao vault
A Bao vault combines the database, an identity, and a storage URL. The storage can be local (`file://`), S3, SFTP, etc.


In [ ]:
realm = "main"
store = Store(local_store('vault', str(temp_root / 'vault')))
vault = Vault.create(realm, bob, db, store)
vault

vault

## Access control
Grant permissions to groups. Built-in groups include `admins`, `users`, and `public`. Permissions are bitmasks (`Access.read`, `Access.write`, `Access.admin`).


In [ ]:
vault.sync_access([
    AccessChange(bob_public, Access.read_write),
])

alice = newPrivateID()
alice_public = publicID(alice)
vault.sync_access([
    AccessChange(alice_public, Access.read),
])

accesses = vault.get_accesses()
for user, access in accesses.items():
    print(f"User {user} has access level: {access}")

print('Alice specific access:', vault.get_access(alice_public))

User A3gXfXOvNYB3g-AzNlT-KQAJD58Cx77UnQbOd191Q5gz-ZBPrJSIMCHTFBd7mU6dNIniOM6t9vC8CvtHLV-IAmk= has access level: 1
User A_1HvChUzrsb93445GF5w8t7a9mlEjMXC0PvfQJTJaL6t4duVKvuW34SNuS3S1Kd9SUAlrkqIX6XxnO4eKtCvCE= has access level: 3


## Open an existing vault
Anyone with the vault URL, their private ID, and the creator's public ID can reopen the vault.


In [ ]:
vault = Vault.open(realm, bob, db, store, {}, bob_public)
vault

vault

## Files and data
Write and read files using group permissions.


In [ ]:
data_path = temp_root / 'hello.txt'
data_path.write_text('Hello from Bao!')

vault.write('docs/hello.txt', src=str(data_path)) # write file to vault
ls = vault.read_dir('docs', None, 0, 10)                        # list files in docs/

print('Files in docs/:', [l.name for l in ls])

Files in docs/: ['hello.txt']


In [9]:
out_path = temp_root / 'hello.out'
vault.read('docs/hello.txt', str(out_path))
print('written content:', out_path.read_text())


written content: Hello from Bao!


I/O operations can be asynchronous by using the option _async_operation_ or _scheduled_operation_

In [ ]:
import time
data_path = temp_root / 'hello.txt'
data_path.write_text('Hello from Bao!')

file = vault.write('docs/hello.txt', src=str(data_path), options=Vault.async_operation) # write file to vault
start = time.perf_counter_ns()

# other operations can be done here while write is in progress
vault.wait_files([file.file_id])      # wait for write to complete
print(f'File {file.name} written in {(time.perf_counter_ns() - start) / 1_000_000 } ms')

File docs/hello.txt written in 1.111459 ms


In [ ]:
file = vault.stat('docs/hello.txt')
print(f'File stats: name = {file.name}', f'size = {file.size} bytes', f'flags = {file.flags}', 
      f'author = {file.author_id}')

#vault.delete('docs/hello.txt')
 

File stats: name = docs/hello.txt size = 15 bytes flags = 0 author =  group = users


## Exchange with another peer
Create a new database to simulate a different peer. Read the content.


In [ ]:
#set_bao_log_level('info')

db2_path = new_db_path('bao2.db')
db2 = DB.open('sqlite3', db2_path)
vault2 = Vault.open(realm, alice, db2, store, {}, bob_public)
vault2.sync()
ls = vault2.read_dir('docs', None, 0, 10)

print('Alice sees files in docs/:', [l['name'] if isinstance(l, dict) else l.name for l in ls])

Alice sees files in docs/: ['hello.txt']


In [11]:
vault2.read('docs/hello.txt', str(temp_root / 'hello.collab.out'))
(temp_root / 'hello.collab.out').read_text()

'Hello from Bao!'

## Messaging
Send and receive `Message` objects inside a vault directory.


In [ ]:
mailbox = Mailbox(vault, 'mailbox')
mailbox.send(Message('Greetings', 'Hello team!'))
mailbox.receive(0, 0)

[Message(subject='Greetings', body='Hello team!', attachments=[], fileInfo={'id': 2, 'name': '5981576e117d000', 'group': 'users', 'size': 0, 'allocatedSize': 456, 'modTime': '2026-01-17T08:58:40.019+01:00', 'isDir': False, 'flags': 0, 'attrs': 'eyJzdWJqZWN0IjoiR3JlZXRpbmdzIiwiYm9keSI6IkhlbGxvIHRlYW0hIiwiYXR0YWNobWVudHMiOltdLCJmaWxlSW5mbyI6eyJpZCI6MCwibmFtZSI6IiIsImdyb3VwIjoiIiwic2l6ZSI6MCwiYWxsb2NhdGVkU2l6ZSI6MCwibW9kVGltZSI6IjAwMDEtMDEtMDFUMDA6MDA6MDBaIiwiaXNEaXIiOmZhbHNlLCJmbGFncyI6MCwia2V5SWQiOjAsInN0b3JlLmlyIjoiIiwic3RvcmUuYW1lIjoiIiwiYXV0aG9ySWQiOiIifX0=', 'keyId': 403095717810786304, 'store.ir': 'data/users/20260117000000', 'store.ame': '3dmqlr1c', 'authorId': 'A49e2k-m1qkRUYJVUSOnsXWQ1sDVRZ5sLepVcxv1HQqwT684t7M2bSkCCBdTw3YSk9P5RrPyNKzRjK211eDm-gE='})]

## Replica interface
Attach a SQL interface that replicates statements through the vault. The interface allows placeholders for the SQL expressions.
The placeholders are defined in the statementes that creates the DB.
Each statements is introduced by a comment:
- a comment starting with _INIT_ and _version_ number is used during the creation of the DB
- other comments are placeholders for the queries

In [ ]:
ddl = '''
-- INIT 1.0
CREATE TABLE IF NOT EXISTS notes (id INTEGER PRIMARY KEY, body TEXT);

-- INSERT_NOTE 1.0
INSERT INTO notes (id, body) VALUES (:id, :body);

-- SELECT_ALL_NOTES 1.0
SELECT * FROM notes;
'''

db_path = temp_root / 'data.db'
if db_path.exists():
    db_path.unlink()
    
data_db = DB.open('sqlite3', str(db_path), ddl=ddl)
replica = Replica(vault, data_db)
replica.exec('INSERT_NOTE', {'id': 1, 'body': 'hello'})
replica.exec('INSERT_NOTE', {'id': 2, 'body': 'bao'})
replica.fetch('SELECT_ALL_NOTES', {})

replica.sync()

2

Another peer can receive the updates

In [ ]:
db_path2 = temp_root / 'data2.db'
if db_path2.exists():
    db_path2.unlink()
data_db2 = DB.open('sqlite3', str(db_path2), ddl=ddl)
replica2 = Replica(vault2, data_db2)
replica2.sync() 
replica2.fetch('SELECT_ALL_NOTES', {})

[[1, 'hello'], [2, 'bao']]